#### IMPORTING ALL NECESSARY FUNCTIONS

In [0]:
from pyspark.sql.functions import col,lit,current_date,date_format,current_timestamp,regexp_extract,row_number,sum,aggregate

from pyspark.sql.window import Window

from pyspark.sql.types import StructType, StructField, StringType, IntegerType,DateType


from pyspark.sql.functions import col, lit, current_date, row_number

from pyspark.sql.window import Window

from datetime import datetime
current_date = datetime.now().strftime("%Y%m%d")


In [0]:

def null_checks(dataframe,column_name_list):
    return dataframe.dropna(subset = column_name_list)


In [0]:

def drop_duplicate_records(dataframe,column_name_list):
    return dataframe.dropDuplicates(subset = column_name_list)


In [0]:

def split_column(dataframe,column_name):
    df = dataframe.withColumn("address1",regexp_extract(col("address"),r"(\d+)",0))\
            .withColumn("address2",regexp_extract(col("address"),r"([^\d]+)",0))

    return df

In [0]:

def apply_scd2(source_df, destination_df, partition_column):
    from pyspark.sql.functions import col, lit, current_date, row_number
    from pyspark.sql.window import Window

    final_df = destination_df.unionByName(source_df)

    window_spec = Window.partitionBy(partition_column).orderBy(col("Created_date").desc())
    ranked_data = final_df.withColumn("row", row_number().over(window_spec))

    active_Record = ranked_data.filter(col("row") == 1).withColumn("Active_status", lit("Yes")).withColumn("END_DATE", lit("9999-12-31"))
    inactive_Record = ranked_data.filter(col("row") > 1).withColumn("Active_status", lit("No")).withColumn("END_DATE", current_date())

    return active_Record.unionByName(inactive_Record)